In [482]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [483]:
#Pesaje Recibido en Puntos Verdes (2015-2016) en kilogramos.
#Importamos
filename = r'C:\Users\leoma\Desktop\UTN\Ciencia de Datos\DataSets\TP2019\pesaje-recibido-en-puntos-verdes-2015-2016.csv'
pesajes = pd.read_csv(filename, delimiter=';', decimal=',')
print(f'Pesajes tiene: {np.shape(pesajes)[0]} filas y {np.shape(pesajes)[0]} columnas')

Pesajes tiene: 3080 filas y 3080 columnas


###  Cuantos puntos y comunas distintas hay?

Mapa de comunas en este [link](https://www.google.com/imgres?imgurl=https%3A%2F%2Fupload.wikimedia.org%2Fwikipedia%2Fcommons%2Fthumb%2F4%2F4b%2FBuenos_Aires_Comunas.svg%2F1200px-Buenos_Aires_Comunas.svg.png&imgrefurl=https%3A%2F%2Fes.wikipedia.org%2Fwiki%2FComunas_de_la_ciudad_de_Buenos_Aires&docid=Ezlwe5ezJGTRlM&tbnid=FSUavRrKMRElQM%3A&vet=10ahUKEwiXpNLmjs7kAhW2HbkGHTRqD2sQMwhRKAAwAA..i&w=1200&h=1222&bih=878&biw=958&q=comunas%20caba&ved=0ahUKEwiXpNLmjs7kAhW2HbkGHTRqD2sQMwhRKAAwAA&iact=mrc&uact=8)

In [484]:
comunas_distintas = np.unique(pesajes.COMUNA)
print(f'Hay {len(comunas_distintas)} comunas distintas')

puntos_distintos = np.unique(pesajes.PUNTO)
print(f'Hay {len(puntos_distintos)} puntos distintos distribuidos entre las comunas')
#pesajes['VIDRIO'][0]

Hay 15 comunas distintas
Hay 32 puntos distintos distribuidos entre las comunas


### Transformamos los meses a valores numéricos

In [485]:
pesajes.loc[ pesajes['MES'] == "ENERO", 'MES'] = 1
pesajes.loc[ pesajes['MES'] == "FEBRERO", 'MES'] = 2
pesajes.loc[ pesajes['MES'] == "MARZO", 'MES'] = 3
pesajes.loc[ pesajes['MES'] == "ABRIL", 'MES'] = 4
pesajes.loc[ pesajes['MES'] == "MAYO", 'MES'] = 5
pesajes.loc[ pesajes['MES'] == "JUNIO", 'MES'] = 6
pesajes.loc[ pesajes['MES'] == "JULIO", 'MES'] = 7
pesajes.loc[ pesajes['MES'] == "AGOSTO", 'MES'] = 8
pesajes.loc[ pesajes['MES'] == "SEPTIEMBRE", 'MES'] = 9
pesajes.loc[ pesajes['MES'] == "OCTUBRE", 'MES'] = 10
pesajes.loc[ pesajes['MES'] == "NOVIEMBRE", 'MES'] = 11
pesajes.loc[ pesajes['MES'] == "DICIEMBRE", 'MES'] = 12

#Separo 2015 y 2016
pesajes_2015 = pesajes.loc[pesajes['ANIO'] == 2015]
pesajes_2016 = pesajes.loc[pesajes['ANIO'] == 2016]

###   Para cada punto, ¿cuantos meses no cargados tienen?         
"No cargado" incluye: 
   * meses que no aparecen en el DF        
   * meses que aparecen pero estan vacíos  
       * "vacíos" son con todo en 0, NaN, "" o " " (string con espacio, no vacío)

Las columnas con NaN, "" o " " solo están en los tipos de material.

Primero cuento los meses por Punto que no aparecen en el DF

In [486]:
## Creo un df de los meses que tengo cargados de cada año, ordenados
meses_cargados_2015 = pesajes_2015.iloc[:,[0,3]].drop_duplicates().sort_values(by=['PUNTO', 'MES'])
meses_cargados_2015 = meses_cargados_2015.copy().reset_index(drop=True)
print(f'\n2015 tiene todos los meses cargados, 32(puntos)x12(meses) = {np.shape(meses_cargados_2015)[0]}\n')

meses_cargados_2016 = pesajes_2016.iloc[:,[0,3]].drop_duplicates().sort_values(by=['PUNTO', 'MES'])
meses_cargados_2016 = meses_cargados_2016.copy().reset_index(drop=True)
print(f'2016 le faltan meses en algunos Puntos, tiene {np.shape(meses_cargados_2016)[0]} meses cargados')
print(f'\nLos numeros de meses que faltan por cada Punto esta guardada en la variable meses_faltantes')

## Creo un array solo de meses, 12 repetidos 32 veces (por cada punto)
Meses_completo_por_Puntos = pd.DataFrame(columns=['MES'])
for x in range(32):
    for mes in range(1,13):
        Meses_completo_por_Puntos.loc[(x*12) + (mes-1)] = mes
Meses_completo_por_Puntos = pd.concat([meses_cargados_2015['PUNTO'], Meses_completo_por_Puntos], axis=1)
    
## Los comparo y obtengo un df de todos los meses faltantes con PUNTO y MES
meses_faltantes = pd.merge(Meses_completo_por_Puntos, meses_cargados_2016,
                           how='outer', indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])

meses_faltantes_por_barrio = meses_faltantes.groupby(["PUNTO"])["MES"].count().sort_values(ascending=False)
    
## Agrego los 2 puntos sin falta de meses para que quede completa la lista
puntos_sin_meses_faltantes = list(filter(lambda x: x not in meses_faltantes_por_barrio.index, puntos_distintos))
meses_faltantes_por_barrio[puntos_sin_meses_faltantes[0]] = 0
meses_faltantes_por_barrio[puntos_sin_meses_faltantes[1]] = 0

Tabla_Faltantes = pd.concat([meses_faltantes_por_barrio], axis=1, keys=['Meses faltantes por barrio en 2016'])
print(f'\nLa variable Tabla_Faltantes tiene la cantidad de meses faltantes por punto para verlos')


2015 tiene todos los meses cargados, 32(puntos)x12(meses) = 384

2016 le faltan meses en algunos Puntos, tiene 298 meses cargados

Los numeros de meses que faltan por cada Punto esta guardada en la variable meses_faltantes

La variable Tabla_Faltantes tiene la cantidad de meses faltantes por punto para verlos


### Segundo cuento los meses que aparecen pero estan vacios, es decir tienen todo 0, "", " " o NaN.

In [487]:
#Primero filtro los que tienen meses con todas las filas llenas de cero (si tienen esas 3 vacias, el resto tambien)
filtro1 = pesajes_2015['VIDRIO'].isin([0,np.nan," "])
filtro2 = pesajes_2015['PAPEL_Y_CARTON'].isin([0,np.nan," "])
filtro3 = pesajes_2015['AVUS'].isin([0,np.nan," "])
sin_vacio = (filtro1 & filtro2 & filtro3)
pesajes_2015_semanas_vacias = pesajes_2015[sin_vacio].drop_duplicates().sort_values(by=['PUNTO', 'MES','SEMANA']).copy().reset_index(drop=True)


filtro1 = pesajes_2016['VIDRIO'].isin([0,np.nan," "])
filtro2 = pesajes_2016['PAPEL_Y_CARTON'].isin([0,np.nan," "])
filtro3 = pesajes_2016['AVUS'].isin([0,np.nan," "])
sin_vacio = (filtro1 & filtro2 & filtro3)
pesajes_2016_semanas_vacias = pesajes_2016[sin_vacio].drop_duplicates().sort_values(by=['PUNTO', 'MES','SEMANA']).copy().reset_index(drop=True)

In [488]:
semanas_faltantes_por_punto_por_mes_2015 = pesajes_2015_semanas_vacias.groupby(["PUNTO", "MES"])["SEMANA"].count().sort_values(ascending=False)

puntos_sin_semanas_faltantes = list(filter(lambda x: x not in semanas_faltantes_por_punto_por_mes_2015.index, puntos_distintos))
print(f'Hay {len(puntos_sin_semanas_faltantes)} puntos sin semanas faltantes')

Tabla_semanas_vacias_2015 = pd.concat([semanas_faltantes_por_punto_por_mes_2015], axis=1, keys=['Semanas faltantes por barrio y mes en 2015'])
print(f'\nLa variable Tabla_semanas_vacias_2015 tiene la cantidad de semanas faltantes por mes por punto para verlos')

print(f'\nEn ella se puede ver que la mayoria de puntos tienen todas las semanas cargadas excepto en su quinta semana de Noviembre o Junio. Esta seguro no es cargada y se toman los meses como de 4 semanas porque en esa semana de esos meses de 2015 hay 2 y 3 dias respectivamente.')
print(f'\nDecidimos eliminar estas quintas semanas vacias y tomar esos meses como de 4 semanas')

Hay 0 puntos sin semanas faltantes

La variable Tabla_semanas_vacias_2015 tiene la cantidad de semanas faltantes por mes por punto para verlos

En ella se puede ver que la mayoria de puntos tienen todas las semanas cargadas excepto en su quinta semana de Noviembre o Junio. Esta seguro no es cargada y se toman los meses como de 4 semanas porque en esa semana de esos meses de 2015 hay 2 y 3 dias respectivamente.

Decidimos eliminar estas quintas semanas vacias y tomar esos meses como de 4 semanas


In [489]:
#Creo el dataframe que va a almacenar las filas que decido borrar
filas_a_borrar_2015 = pesajes_2015_semanas_vacias.head(1).copy()
filas_a_borrar_2015 = filas_a_borrar_2015.drop([0], axis=0)

pesajes_2015_semanas_vacias_2 = pesajes_2015_semanas_vacias.copy()
indices = []

for indice in range(153):
    if (((pesajes_2015_semanas_vacias_2.iloc[indice]['MES'] == 6) | (pesajes_2015_semanas_vacias_2.iloc[indice]['MES'] == 11)) & (pesajes_2015_semanas_vacias_2.iloc[indice]['SEMANA'] == 5)):
        filas_a_borrar_2015 = pd.concat([filas_a_borrar_2015, pesajes_2015_semanas_vacias_2.iloc[indice].copy()])
        indices.append(indice)
pesajes_2015_semanas_vacias_2 = pesajes_2015_semanas_vacias_2.drop(indices, axis=0)

#Vuelvo a crear la tabla, ahora filtrada sin las quintas semanas de Noviembre y Junio
meses_faltantes_por_punto_por_mes_2015_filtrado = pesajes_2015_semanas_vacias_2.groupby(["PUNTO", "MES"])["SEMANA"].count()
Tabla_semanas_vacias_filtrada_2015 = pd.concat([meses_faltantes_por_punto_por_mes_2015_filtrado], axis=1, keys=['Semanas faltantes por barrio y mes en 2015, sin quintas semanas de mas'])

In [490]:
#Ahora hago lo mismo pero para el año 2016
meses_faltantes_por_punto_por_mes_2016 = pesajes_2016_semanas_vacias.groupby(["PUNTO", "MES"])["SEMANA"].count().sort_values(ascending=False)

puntos_sin_semanas_faltantes = list(filter(lambda x: x not in meses_faltantes_por_punto_por_mes_2016.index, puntos_distintos))
print(f'Hay {len(puntos_sin_semanas_faltantes)} puntos sin semanas faltantes')

Tabla_semanas_vacias_2016 = pd.concat([meses_faltantes_por_punto_por_mes_2016], axis=1, keys=['Semanas faltantes por barrio y mes en 2015'])

print(f'\nLa variable Tabla_semanas_vacias_2016 tiene la cantidad de semanas faltantes por mes por punto para verlos')

print(f'\nEn ella se puede ver que la mayoria de puntos tienen todas las semanas cargadas excepto en su quinta semana de Enero. Problema parecido al de 2015')
print(f'\nDecidimos eliminar estas quintas semanas vacias y tomar este mes como de 4 semanas')

print(f'\nSe ve que en la mayoria de barrios las semanas 4 y 5 de Septiembre no estan cargadas')

Hay 0 puntos sin semanas faltantes

La variable Tabla_semanas_vacias_2016 tiene la cantidad de semanas faltantes por mes por punto para verlos

En ella se puede ver que la mayoria de puntos tienen todas las semanas cargadas excepto en su quinta semana de Enero. Problema parecido al de 2015

Decidimos eliminar estas quintas semanas vacias y tomar este mes como de 4 semanas

Se ve que en la mayoria de barrios las semanas 4 y 5 de Septiembre no estan cargadas


In [491]:
#Creo el dataframe que va a almacenar las filas que decido borrar
filas_a_borrar_2016 = pesajes_2016_semanas_vacias.head(1).copy()
filas_a_borrar_2016 = filas_a_borrar_2016.drop([0], axis=0)

pesajes_2016_semanas_vacias_2 = pesajes_2016_semanas_vacias.copy()
indices = []

for indice in range(179):
    if ((pesajes_2016_semanas_vacias_2.iloc[indice]['MES'] == 1) & (pesajes_2016_semanas_vacias_2.iloc[indice]['SEMANA'] == 5)):
        filas_a_borrar_2016 = pd.concat([filas_a_borrar_2016, pesajes_2016_semanas_vacias_2.iloc[indice].copy()])
        indices.append(indice)
pesajes_2016_semanas_vacias_2 = pesajes_2016_semanas_vacias_2.drop(indices, axis=0)

#Vuelvo a crear la tabla, ahora filtrada sin las quintas semanas de Noviembre y Junio
meses_faltantes_por_punto_por_mes_2016_filtrado = pesajes_2016_semanas_vacias_2.groupby(["PUNTO", "MES"])["SEMANA"].count()
Tabla_semanas_vacias_filtrada_2016 = pd.concat([meses_faltantes_por_punto_por_mes_2016_filtrado], axis=1, keys=['Semanas faltantes por barrio y mes en 2016, sin quintas semanas de mas'])


### Ahora tomamos como criterio: 
    Si un mes tiene 4-5 semanas vacias entonces esta vacio

### Unimos las tablas creadas de 2015 y 2016 para ver los resultados


In [492]:
Tabla_de_Vacios_Completa = meses_faltantes.groupby(["PUNTO"])["MES"].count().sort_values(ascending=False)
    
## Agrego los 2 puntos sin falta de meses para que quede completa la lista
puntos_sin_meses_faltantes_2 = list(filter(lambda x: x not in Tabla_de_Vacios_Completa.index, puntos_distintos))
Tabla_de_Vacios_Completa[puntos_sin_meses_faltantes_2[0]] = 0
Tabla_de_Vacios_Completa[puntos_sin_meses_faltantes_2[1]] = 0

Tabla_de_Vacios_Completa = Tabla_de_Vacios_Completa.to_frame().rename(columns = {"MES": "Faltantes_2016"})

In [493]:
#Salvo por marzo en PLAZA CASTELLI, los demas meses los consideramos todos vacios segun el criterio
#pesajes_2016_semanas_vacias_2 = pesajes_2016_semanas_vacias_2.iloc[:,[0,3]].drop_duplicates().sort_values(by=['PUNTO'])
#pesajes_2016_semanas_vacias_2

#Marco con 1 los meses que tienen 4 o mas semanas faltantes
Agregar_a_tabla_vacias = pesajes_2016_semanas_vacias_2.groupby(["PUNTO", "MES"]).filter(lambda x: x["SEMANA"].count() >= 3)
Agregar_a_tabla_vacias_filtrada = Agregar_a_tabla_vacias.iloc[:,[0,3]].drop_duplicates().sort_values(by=['PUNTO', 'MES'])
Agregar_a_tabla_vacias_filtrada = Agregar_a_tabla_vacias_filtrada.groupby(["PUNTO"]).size().to_frame().rename(columns = {0: "Vacios_2016"})

Agregar_a_tabla_vacias_2015 = pesajes_2015_semanas_vacias.groupby(["PUNTO", "MES"]).filter(lambda x: x["SEMANA"].count() >= 3)
Agregar_a_tabla_vacias_filtrada_2015 = Agregar_a_tabla_vacias_2015.iloc[:,[0,3]].drop_duplicates().sort_values(by=['PUNTO', 'MES'])
Agregar_a_tabla_vacias_filtrada_2015 = Agregar_a_tabla_vacias_filtrada_2015.groupby(["PUNTO"]).size().to_frame().rename(columns = {0: "Meses_no_utilizables_2015"})

Tabla_de_Vacios_Completa = pd.merge(Tabla_de_Vacios_Completa, Agregar_a_tabla_vacias_filtrada, how='left', on='PUNTO').fillna(0)
Tabla_de_Vacios_Completa['Meses_no_utilizables_2016'] = Tabla_de_Vacios_Completa.Faltantes_2016 + Tabla_de_Vacios_Completa.Vacios_2016
Tabla_de_Vacios_Completa.sort_values(by=['Meses_no_utilizables_2016'],ascending=False)

Tabla_de_Vacios_Completa = pd.merge(Tabla_de_Vacios_Completa, Agregar_a_tabla_vacias_filtrada_2015, how='left', on='PUNTO').fillna(0)